In [13]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate
import cv2
import pickle
from scipy.ndimage import median_filter, convolve
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
ls '/content/drive/My Drive/'

'CAE Result  - C1   p.12.pdf'
'Colab Notebooks'/
'CV Joanna Bilicka (Recovered).gdoc'
 CWB-C.gform
'CWB-C (Responses).gsheet'
 flowers/
 foo.txt
'Grammar & Vocabulary for Cambridge Advanced & Proficiency p. 2.pdf'
'How to get started with Drive.pdf'
'kryteria .gsheet'
'NetGuru task.docx'
'NetGuru task.gdoc'
'Nowy folder.rar'
'other shit.gdoc'
'Plan odżywiania Bartek trening po południu.gdoc'
'Plan odżywiania Bartek trening po południu.odt'
'Plan odżywiania Bartek trening po południu.odt.gdoc'
'Testy Pan Bartosz.docx'
'Testy Pan Bartosz.docx.gdoc'
'Untitled form.gform'
'Untitled form (Responses).gsheet'
 Wykladowca_hojnacki_bartosz_dzienne_zimowy_18_19.gsheet
 Wykladowca_hojnacki_bartosz_dzienne_zimowy_18_19.xls
'Wyliczenie netto na działalności gospodarczej i umowie o świadczenie usług dla Bartosza Hojnackiego.xlsx'
'Wyliczenie netto na działalności gospodarczej i umowie o świadczenie usług dla Bartosza Hojnackiego.xlsx.gsheet'


In [7]:
data_dir = "/content/drive/My Drive/flowers"

categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [8]:
data = []
def make_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)
        
        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)
            
            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224,224))
                
                image = np.array(image, dtype=np.float32)
                
                data.append([image, label])
            except Exception as e:
                pass
    print(len(data))
    
    pik = open('data.pickle', 'wb')
    pickle.dump(data,pik)
    pik.close()

In [9]:
make_data()

4323


In [10]:

def load_data():
    pick = open('data.pickle','rb')
    data = pickle.load(pick)
    pick.close()
    
    np.random.shuffle(data)
    
    feature = []
    labels = []
    
    for img, label in data:
        feature.append(img)
        labels.append(label)
        
    feature = np.array(feature, dtype=np.float32)
    labels = np.array(labels)
    
    feature = feature/255.0
    
    return [feature, labels]

In [11]:
(feature, labels) = load_data()

In [14]:
x_train, x_test, y_train, y_test = train_test_split (feature, labels, test_size = 0.1)

In [15]:
input_shape = (224,224,3)

In [23]:
model = keras.applications.MobileNetV2(weights='imagenet', input_shape=input_shape, include_top=True)

14540800/14536120 [==============================] - 0s 0us/step


In [18]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [24]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"])

In [26]:
batch_size = 100
epochs = 20
patience = 5
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), 
          callbacks = [tf.keras.callbacks.EarlyStopping(patience=patience)])

Epoch 1/20
39/39 [==============================] - 25s 653ms/step - loss: 0.0722 - accuracy: 0.9776 - val_loss: 6.5063 - val_accuracy: 0.5566
Epoch 2/20
39/39 [==============================] - 25s 635ms/step - loss: 0.0654 - accuracy: 0.9774 - val_loss: 16.2709 - val_accuracy: 0.3880
Epoch 3/20
39/39 [==============================] - 24s 622ms/step - loss: 0.0653 - accuracy: 0.9781 - val_loss: 11.0983 - val_accuracy: 0.4550
Epoch 4/20
39/39 [==============================] - 24s 625ms/step - loss: 0.0374 - accuracy: 0.9887 - val_loss: 3.7398 - val_accuracy: 0.6674
Epoch 5/20
39/39 [==============================] - 25s 631ms/step - loss: 0.0559 - accuracy: 0.9830 - val_loss: 3.2359 - val_accuracy: 0.6328
Epoch 6/20
39/39 [==============================] - 24s 625ms/step - loss: 0.0372 - accuracy: 0.9877 - val_loss: 5.0822 - val_accuracy: 0.5982
Epoch 7/20
39/39 [==============================] - 25s 631ms/step - loss: 0.0428 - accuracy: 0.9848 - val_loss: 12.9153 - val_accuracy: 0.5

In [27]:
model.save('outdated_mobilenetv2.h5')

In [28]:
from google.colab import files
files.download('outdated_mobilenetv2.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
model.evaluate(x_test, y_test, verbose = 1)

14/14 [==============================] - 1s 40ms/step - loss: 11.4805 - accuracy: 0.4018


[11.480475425720215, 0.40184757113456726]